In [3]:

from mols2grid import display
from torch.nn.functional import linear

# Přidání cesty k adresáři, kde se nachází tvůj skript
# sys.path.append(os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/qsprpred/extra/gpu/models'))
import sys, os
sys.path.insert(0, os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/'))

import pandas as pd

df = pd.read_csv('../../../../tutorials/tutorial_data/A2A_LIGANDS.tsv', sep='\t')

df.head()
from qsprpred.data import QSPRDataset

import os

os.makedirs("tutorial_output/data", exist_ok=True)

dataset = QSPRDataset(
    df=df,
    store_dir="tutorial_output/data",
    name="QuickStartDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42,
    overwrite=True
)

dataset.getDF()
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data import RandomSplit


# Specifiy random split for creating the train (80%) and test set (20%)
rand_split = RandomSplit(test_fraction=0.2, dataset=dataset)

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=rand_split,
    feature_calculators=[MorganFP(radius=3, nBits=2048)],
)

print(f"Number of samples in train set: {len(dataset.y)}")
print(f"Number of samples in test set: {len(dataset.y_ind)}")

dataset.save()

import torch

from torch.nn import functional as f
from sklearn.metrics import mean_squared_error
from qsprpred.extra.gpu.models.neural_network import STFullyConnected


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = STFullyConnected(
        n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
        n_class=1,  # regresní úloha (1 výstup)
        gpus=[],
        device=device,
        batch_size=256,
        patience=20,
        tol=1e-4,
        is_reg=True,
        extra_layer=True,
        act_fun=f.selu,
        neuron_layers=[2048, 1024, 512],
    )
x.fit(dataset.X, dataset.y)

print(mean_squared_error(dataset.y_ind, x.predict(dataset.X_ind)))

Number of samples in train set: 3265
Number of samples in test set: 817
[2048, 1024, 512]
512
ModuleList(
  (0): Linear(in_features=2048, out_features=2048, bias=True)
  (1): Linear(in_features=2048, out_features=1024, bias=True)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): Linear(in_features=512, out_features=1, bias=True)
)
Linear(in_features=2048, out_features=256, bias=True) Linear(in_features=256, out_features=128, bias=True) Linear(in_features=128, out_features=128, bias=True) Linear(in_features=128, out_features=1, bias=True)
0.5879974768899853


TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [1]:

from mols2grid import display
from torch.nn.functional import linear

# Přidání cesty k adresáři, kde se nachází tvůj skript
# sys.path.append(os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/qsprpred/extra/gpu/models'))
import sys, os
sys.path.insert(0, os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/'))
# Nyní můžeš naimportovat svůj skript
print(sys.path)


['/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred', '/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred', '/Users/krynekt/anaconda3/envs/ml/lib/python311.zip', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/lib-dynload', '', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/site-packages', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/site-packages/setuptools/_vendor']


In [2]:
import pandas as pd

df = pd.read_csv('../../../../tutorials/tutorial_data/A2A_LIGANDS.tsv', sep='\t')

df.head()

,SMILES,pchembl_value_Mean,Year
0,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,8.68,2008.0
1,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2c1cc1CCCC...,4.82,2010.0
2,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,2009.0
3,CNC(=O)C12CC1C(n1cnc3c1nc(C#CCCCCC(=O)OC)nc3NC...,5.45,2009.0
4,CCCn1c(=O)c2c(nc3cc(OC)ccn32)n(CCCNC(=O)c2ccc(...,5.20,2019.0


In [3]:
from qsprpred.data import QSPRDataset

import os

os.makedirs("tutorial_output/data", exist_ok=True)

dataset = QSPRDataset(
    df=df,
    store_dir="tutorial_output/data",
    name="QuickStartDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42,
    overwrite=True
)

dataset.getDF()

,SMILES,pchembl_value_Mean,Year,QSPRID,pchembl_value_Mean_original
QSPRID,,,,,
QuickStartDataset_0000,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,8.68,2008.0,QuickStartDataset_0000,8.68
QuickStartDataset_0001,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2c1cc1CCCC...,4.82,2010.0,QuickStartDataset_0001,4.82
QuickStartDataset_0002,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,2009.0,QuickStartDataset_0002,5.65
QuickStartDataset_0003,CNC(=O)C12CC1C(n1cnc3c1nc(C#CCCCCC(=O)OC)nc3NC...,5.45,2009.0,QuickStartDataset_0003,5.45
QuickStartDataset_0004,CCCn1c(=O)c2c(nc3cc(OC)ccn32)n(CCCNC(=O)c2ccc(...,5.20,2019.0,QuickStartDataset_0004,5.20
...,...,...,...,...,...
QuickStartDataset_4077,CNc1ncc(C(=O)NCc2ccc(OC)cc2)c2nc(-c3ccco3)nn12,7.09,2018.0,QuickStartDataset_4077,7.09
QuickStartDataset_4078,Nc1nc(-c2ccco2)c2ncn(C(=O)NCCc3ccccc3)c2n1,8.22,2008.0,QuickStartDataset_4078,8.22
QuickStartDataset_4079,Nc1nc(Nc2ccc(F)cc2)nc(CSc2nnc(N)s2)n1,4.89,2010.0,QuickStartDataset_4079,4.89


In [4]:
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data import RandomSplit

# Specifiy random split for creating the train (80%) and test set (20%)
rand_split = RandomSplit(test_fraction=0.2, dataset=dataset)

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=rand_split,
    feature_calculators=[MorganFP(radius=3, nBits=2048)],
)

print(f"Number of samples in train set: {len(dataset.y)}")
print(f"Number of samples in test set: {len(dataset.y_ind)}")

dataset.save()

Number of samples in train set: 3265
Number of samples in test set: 817


In [5]:

print(dataset.getDescriptors())
# display(dataset.X.head())

                        MorganFP_MorganFP_0  MorganFP_MorganFP_1  \
QSPRID                                                             
QuickStartDataset_0000                False                False   
QuickStartDataset_0001                False                False   
QuickStartDataset_0002                False                False   
QuickStartDataset_0003                False                False   
QuickStartDataset_0004                False                False   
...                                     ...                  ...   
QuickStartDataset_4077                False                False   
QuickStartDataset_4078                False                False   
QuickStartDataset_4079                False                False   
QuickStartDataset_4080                False                False   
QuickStartDataset_4081                False                 True   

                        MorganFP_MorganFP_2  MorganFP_MorganFP_3  \
QSPRID                                         

In [6]:
import torch
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score
from torch.nn import functional as f
from sklearn.metrics import mean_squared_error
from qsprpred.extra.gpu.models.neural_network import STFullyConnected


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# p = {"n_epochs": [200],
#                             "lr": [1e-4],
#                             "neurons_h1": [2048],
#                             "neurons_hx": [1024],
#                             "weight_decay": [1e-4],
#                             "dropout_frac": [0.0],
#                             "act_fun": [f.selu]}
x = STFullyConnected(
        n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
        n_class=1,  # regresní úloha (1 výstup)
        gpus=[],
        device=device,
        batch_size=256,
        patience=20,
        tol=1e-4,
        is_reg=True,
        extra_layer=False,
        act_fun=f.selu,
    )
x.fit(dataset.X, dataset.y)
print(mean_squared_error(dataset.y_ind, x.predict(dataset.X_ind)))

test
[Linear(in_features=2048, out_features=256, bias=True), Linear(in_features=256, out_features=128, bias=True), Linear(in_features=128, out_features=1, bias=True)]
Linear(in_features=2048, out_features=256, bias=True) Linear(in_features=256, out_features=128, bias=True) None Linear(in_features=128, out_features=1, bias=True)
Input shape: torch.Size([256, 2048])
Layer 0 input shape: torch.Size([256, 2048])
Layer 0 output shape: torch.Size([256, 256])
Layer 1 input shape: torch.Size([256, 256])
Layer 1 output shape: torch.Size([256, 128])
Layer 2 input shape: torch.Size([256, 128])
Layer 2 output shape: torch.Size([256, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x1 and 128x1)

In [7]:

test.fit(dataset.X, dataset.y)
# print(mean_squared_error(test.predict(dataset.X_ind), dataset.y_ind))

NameError: name 'test' is not defined

In [7]:
print(dataset.X.shape[1])
print(dataset.hasFeatures)

2048
True


In [10]:
display(dataset.X.head())

MolGridWidget()

KeyError: "['SMILES'] not in index"

In [12]:




param_grid = ParameterGrid({"n_epochs": [200],
                            "lr": [1e-4],
                            "neurons_h1": [2048],
                            "neurons_hx": [1024],
                            "weight_decay": [1e-4],
                            "dropout_frac": [0.0],
                            "act_fun": [f.selu]})  #selu
val_accuracies = []
val_accuracies_r = []
ind = 0

for params in param_grid:
    ind += 1
    print(ind, '/', len(param_grid))
    model_nn = STFullyConnected(
        n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
        n_class=1,  # regresní úloha (1 výstup)
        gpus=[],
        device=device,
        batch_size=256,
        patience=20,
        tol=1e-4,
        is_reg=True,
        extra_layer=True,
        **params
    )
    print(model_nn.get_params())
    model_nn.fit(dataset.X, dataset.y)
    print(params["act_fun"], mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
    val_accuracies.append(mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
    val_accuracies_r.append(r2_score(model_nn.predict(dataset.X_ind), dataset.y_ind))









1 / 1
[Linear(in_features=2048, out_features=256, bias=True), Linear(in_features=256, out_features=128, bias=True), Linear(in_features=128, out_features=1, bias=True)]
Linear(in_features=2048, out_features=2048, bias=True) Linear(in_features=2048, out_features=1024, bias=True) Linear(in_features=1024, out_features=1024, bias=True) Linear(in_features=1024, out_features=1, bias=True)
{'act_fun': <function selu at 0x13300a160>, 'batch_size': 256, 'device': device(type='cpu'), 'dropout_frac': 0.0, 'extra_layer': True, 'gpus': [], 'is_reg': True, 'lr': 0.0001, 'n_class': 1, 'n_dim': 2048, 'n_epochs': 200, 'neuron_layers': [256, 128], 'neurons_h1': 2048, 'neurons_hx': 1024, 'patience': 20, 'tol': 0.0001, 'weight_decay': 0.0001}


KeyboardInterrupt: 

In [12]:
print(val_accuracies)
print(val_accuracies_r)

[0.591590194764539]
[0.44612061891762367]


In [21]:
import numpy as np

best_params = param_grid[(np.argmin(val_accuracies))]
display(best_params)
display(min(val_accuracies))
display(val_accuracies)


ValueError: If using all scalar values, you must pass an index


* RELU 1.045242449663438
* GELU 1.012
* ELU 1.0803313948254016
* SiLU 1.054
* SELU 1.089
* Dropout: 0.05


1 / 8
* <function relu at 0x15eb3a160> 1.1250744327450628
2 / 8
* <function leaky_relu at 0x15eb3a5c0> 1.0938768673113628
3 / 8
* <built-in function gelu> 1.1638879556715451
4 / 8
* <function elu at 0x15eb3a3e0> 0.9410014660390317
5 / 8
* <function silu at 0x15eb3ad40> 1.264585029437087
6 / 8
* <function tanh at 0x15eb3ab60> 1.5572187279142176
7 / 8
* <function sigmoid at 0x15eb3ac00> 9.547899011065818
8 / 8
* <function selu at 0x15eb3a480> 0.7631453763567609


Base model:
* MSE <- 0.31
* R2 <- 0.68


In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

model_nn = STFullyConnected(n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
                            n_class=1,  # regresní úloha (1 výstup)
                            gpus=[],
                            device=device,
                            batch_size=256,
                            patience=20,
                            tol=1e-4,
                            is_reg=True,
                            extra_layer=True,
                            dropout_frac=0.25,
                            **best_params)

print("MSE:", mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
print("R2:", r2_score(model_nn.predict(dataset.X_ind), dataset.y_ind))


NameError: name 'best_params' is not defined

In [20]:
model_nn_noreg = STFullyConnected(
    n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
    n_class=1,  # regresní úloha (1 výstup)
    gpus=[],
    device=device,
    n_epochs=200,
    lr=1e-4,
    batch_size=256,
    patience=20,
    tol=1e-4,
    is_reg=True,
    neurons_h1=1000,
    neurons_hx=200,
    extra_layer=True,
    dropout_frac=0.25,
    weight_decay=0.0
)


In [21]:
model_nn_noreg.fit(dataset.X, dataset.y)

(STFullyConnected(
   (dropout): Dropout(p=0.25, inplace=False)
   (fc0): Linear(in_features=2048, out_features=1000, bias=True)
   (fc1): Linear(in_features=1000, out_features=200, bias=True)
   (fc2): Linear(in_features=200, out_features=200, bias=True)
   (fc3): Linear(in_features=200, out_features=1, bias=True)
   (criterion): MSELoss()
 ),
 0)

In [22]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print("MSE:", mean_squared_error(model_nn_noreg.predict(dataset.X_ind), dataset.y_ind))
print("R2:", r2_score(model_nn_noreg.predict(dataset.X_ind), dataset.y_ind))

MSE: 0.7215939088954494
R2: 0.279348695180174
